In [1]:

from typing import Literal, List

_BACKEND = None
PYQUDA = None


def get_backend():
    global _BACKEND
    if _BACKEND is None:
        set_backend("numpy")
    return _BACKEND


def set_backend(backend: Literal["numpy", "cupy"]):
    global _BACKEND
    if not isinstance(backend, str):
        backend = backend.__name__
    backend = backend.lower()
    assert backend in ["numpy", "cupy"]
    if backend == "numpy":
        import numpy

        _BACKEND = numpy
    elif backend == "cupy":
        import cupy

        _BACKEND = cupy
    # elif backend == "torch":
    #     import torch
    #     torch.set_default_device("cuda")
    #     _BACKEND = torch
    else:
        raise ValueError(R'backend must be "numpy", "cupy" or "torch"')

In [8]:
from functools import lru_cache
import numpy 

class _Constant:
    @staticmethod
    @lru_cache(1)
    def zero():
        return numpy.zeros((4, 4))

    @staticmethod
    @lru_cache(1)
    def one():
        return numpy.identity(4)

    @staticmethod
    @lru_cache(1)
    def gamma_0():
        return numpy.array(
            [
                [0, 0, 0, 1j],
                [0, 0, 1j, 0],
                [0, -1j, 0, 0],
                [-1j, 0, 0, 0],
            ]
        )

    @staticmethod
    @lru_cache(1)
    def gamma_1():
        return numpy.array(
            [
                [0, 0, 0, -1],
                [0, 0, 1, 0],
                [0, 1, 0, 0],
                [-1, 0, 0, 0],
            ]
        )

    @staticmethod
    @lru_cache(1)
    def gamma_2():
        return numpy.array(
            [
                [0, 0, 1j, 0],
                [0, 0, 0, -1j],
                [-1j, 0, 0, 0],
                [0, 1j, 0, 0],
            ]
        )

    @staticmethod
    @lru_cache(1)
    def gamma_3():
        return numpy.array(
            [
                [0, 0, 1, 0],
                [0, 0, 0, 1],
                [1, 0, 0, 0],
                [0, 1, 0, 0],
            ]
        )

# below is copying the chroma matrix conventions from the documentation #

def gamma(n: int):
    assert isinstance(n, int) and n >= 0 and n <= 15
    return numpy.asarray(
        (_Constant.gamma_0() if n & 0b0001 else _Constant.one())
        @ (_Constant.gamma_1() if n & 0b0010 else _Constant.one())
        @ (_Constant.gamma_2() if n & 0b0100 else _Constant.one())
        @ (_Constant.gamma_3() if n & 0b1000 else _Constant.one())
    )
_naming_scheme = {
    "a0": [[0]],
    "a0(2)": [[8]],
    "pi": [[15]],
    "pi(2)": [[8, 15]],
    "rho": [[1], [2], [4]],
    "rho(2)": [[8, 1], [8, 2], [8, 4]],
    "a1": [[15, 1], [15, 2], [15, 4]],
    "b1": [[8, 15, 1], [8, 15, 2], [8, 15, 4]],
}


def gamma_scheme(name: str):
    assert name in _naming_scheme
    return _naming_scheme[name]


class GAMMA_NAME:
    A0 = "a0"
    A0_2 = "a0(2)"
    PI = "pi"
    PI_2 = "pi(2)"
    RHO = "rho"
    RHO_2 = "rho(2)"
    A1 = "a1"
    B1 = "b1"


def instance(gamma_idxs: list):
    ret = _Constant.one()
    for idx in gamma_idxs:
        ret = ret @ gamma(idx)
    return ret
# for _name in _naming_scheme.values():
for i in range(0,15):
    print(i,gamma(i),'\n')

0 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 

1 [[0.+0.j 0.+0.j 0.+0.j 0.+1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]] 

2 [[ 0.  0.  0. -1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [-1.  0.  0.  0.]] 

3 [[0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j]] 

4 [[0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]] 

5 [[ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]] 

6 [[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]] 

7 [[ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]] 

8 [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 

In [13]:
from typing import List, Union


class DerivPart:
    def __init__(self, coeff: float, deriv: str) -> None:
        self.coeff: float = coeff
        self.deriv: Union[int, str] = deriv

    def __repr__(self) -> str:
        return f"{self.coeff:.3f} * {self.deriv}"

    def normalize(self, sumsq):
        self.coeff *= sumsq**-0.5

d = DerivPart(coeff=1.2,deriv="1 1")


None


In [ ]:


class Deriv:
    def __init__(self, parts) -> None:
        sumsq = 0
        self.parts: List[DerivPart] = []
        for part in parts:
            sumsq += part[0]**2
            self.parts.append(DerivPart(part[0], part[1]))
        for part in self.parts:
            part.normalize(sumsq)

    def __repr__(self) -> str:
        ret = [part for part in self.parts]
        return f"{ret}"

''' see Appendix B of https://arxiv.org/pdf/0707.4162; the 2nd and 3rd rows of table '''

disp_map = {
    # zero displacement 
    "_": [
        Deriv([[1, ""]]),
    ],

    # single-derivative level #
    "d": [
        Deriv([[1, "1"]]),
        Deriv([[1, "2"]]),
        Deriv([[1, "3"]]),
    ],

    # two derivative level #
    # D: |eps_ijk| 
    # B : eps_ijk
    # Q: Cg coeff 
    #: Laplacian^2 

    "D": [
        Deriv([[1, "2 3"], [1, "3 2"]]),
        Deriv([[1, "3 1"], [1, "1 3"]]),
        Deriv([[1, "1 2"], [1, "2 1"]]),
    ],
    "B": [
        Deriv([[1, "2 3"], [-1, "3 2"]]),
        Deriv([[1, "3 1"], [-1, "1 3"]]),
        Deriv([[1, "1 2"], [-1, "2 1"]]),
    ],

    "Q": [
        Deriv([[1, "1 1"], [-1, "2 2"]]),
        Deriv([[-1, "1 1"], [-1, "2 2"], [2, "3 3"]]),
    ],
}



def deriv_scheme(name: str):
    assert name in _naming_scheme
    return _naming_scheme[name]


class DERIV_NAME:
    pass

In [8]:
from typing import List



class OperatorPart:
    def __init__(self, coeff: int, gamma, deriv) -> None:

        self.coeff: int = coeff
        self.gamma: List[int] = gamma
        self.deriv: Deriv = deriv

    def normalize(self, sumsq: int):
        for part in self.deriv.parts:
            part.coeff *= self.coeff * sumsq ** -0.5
        self.coeff = 1

    def __repr__(self) -> str:
        return f"{self.gamma} {self.deriv}"


class Operator:
    def __init__(self, parts: list) -> None:
        from copy import deepcopy as cp

        sumsq = 0
        self.parts: List[OperatorPart] = []
        for part in parts:
            sumsq += part[0] ** 2
            self.parts.append(OperatorPart(part[0], part[1], cp(part[2])))
        for part in self.parts:
            part.normalize(sumsq)

    def __repr__(self) -> str:
        ret = [part for part in self.parts]
        return f"{ret}"


def only(gamma_name: GAMMA_NAME):

    gamma = gamma_scheme(gamma_name)
    deriv = deriv_scheme("_")
    return [
        Operator(
            [
                [1, gamma[0], deriv[0]],
            ]
        ),
        Operator(
            [
                [1, gamma[1], deriv[0]],
            ]
        ),
        Operator(
            [
                [1, gamma[2], deriv[0]],
            ]
        ),
    ]


def multiply(gamma_name: GAMMA_NAME, deriv_name: DERIV_NAME):
    gamma = gamma_scheme(gamma_name)
    deriv = deriv_scheme(deriv_name)
    return [
        Operator(
            [
                [1, gamma[0], deriv[0]],
            ]
        ),
        Operator(
            [
                [1, gamma[0], deriv[1]],
            ]
        ),
        Operator(
            [
                [1, gamma[0], deriv[2]],
            ]
        ),
    ]


def dot(gamma_name: GAMMA_NAME, deriv_name: DERIV_NAME):
    gamma = gamma_scheme(gamma_name)
    deriv = deriv_scheme(deriv_name)
    return [
        Operator(
            [
                [1, gamma[0], deriv[0]],
                [1, gamma[1], deriv[1]],
                [1, gamma[2], deriv[2]],
            ]
        )
    ]


def epsilon_ijk(gamma_name: GAMMA_NAME, deriv_name: DERIV_NAME):
    gamma = gamma_scheme(gamma_name)
    deriv = deriv_scheme(deriv_name)
    return [
        Operator(
            [
                [1, gamma[1], deriv[2]],
                [-1, gamma[2], deriv[1]],
            ]
        ),
        Operator(
            [
                [1, gamma[2], deriv[0]],
                [-1, gamma[0], deriv[2]],
            ]
        ),
        Operator(
            [
                [1, gamma[0], deriv[1]],
                [-1, gamma[1], deriv[0]],
            ]
        ),
    ]


def abs_epslion_ijk(gamma_name: GAMMA_NAME, deriv_name: DERIV_NAME):
    gamma = gamma_scheme(gamma_name)
    deriv = deriv_scheme(deriv_name)
    return [
        Operator(
            [
                [1, gamma[1], deriv[2]],
                [1, gamma[2], deriv[1]],
            ]
        ),
        Operator(
            [
                [1, gamma[2], deriv[0]],
                [1, gamma[0], deriv[2]],
            ]
        ),
        Operator(
            [
                [1, gamma[0], deriv[1]],
                [1, gamma[1], deriv[0]],
            ]
        ),
    ]


def Q_ijk(gamma_name: GAMMA_NAME, deriv_name: DERIV_NAME):
    gamma = gamma_scheme(gamma_name)
    deriv = deriv_scheme(deriv_name)
    return [
        Operator(
            [
                [1, gamma[0], deriv[0]],
                [-1, gamma[1], deriv[1]],
            ]
        ),
        Operator(
            [
                [-1, gamma[0], deriv[0]],
                [-1, gamma[1], deriv[1]],
                [2, gamma[2], deriv[2]],
            ]
        ),
    ]
gamma_name = GAMMA_NAME.PI # Example gamma name
derivDict = {
    "_": 0,
    "another_deriv": 1,
    # More mappings can be added as needed
}
deriv_name = deriv_scheme("_") # Example derivative name

# Create a list of operators using the `dot` helper function
operators = dot(gamma_name, "_")

AssertionError: 